In [1]:
from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 모델 경로와 토크나이저 로드
model_path = '/DATA/hub/models--defog--llama-3-sqlcoder-8b/snapshots/0f96d32e16737bda1bbe0d8fb13a932a8a3fa0bb'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# 파이프라인 객체 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=1,  # GPU 사용, CPU는 -1
    model_kwargs={"temperature": 0.0, "max_length": 200}
)

# HuggingFacePipeline 객체 생성
llm = HuggingFacePipeline(pipeline=pipe)

# 프롬프트 템플릿 정의
template = """질문: {question}

주의사항: 아래의 답변은 SQL 쿼리문으로만 작성됩니다. 다른 설명이나 답변 없이 SQL 쿼리문만 생성해 주세요.

SQL 답변: """
prompt = PromptTemplate.from_template(template)

# LLM Chain 객체 생성
llm_chain = LLMChain(prompt=prompt, llm=llm)

/DATA/acorn/anaconda3/envs/LangChain03Env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.49s/it]
/DATA/acorn/anaconda3/envs/LangChain03Env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/DATA/acorn/anaconda3/envs/LangChain03Env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set

In [7]:

# 예시 질문 실행
question = """
Table Schema

events (
    event_date TEXT PRIMARY KEY, -- 이벤트가 발생한 날짜
    event_timestamp INTEGER, -- 이벤트 타임스탬프
    event_name TEXT, -- 이벤트 이름
    event_params BLOB, -- 이벤트 파라미터 (JSON 데이터)
    event_previous_timestamp INTEGER, -- 이전 이벤트 타임스탬프
    event_value_in_usd REAL, -- 이벤트의 달러 기준 값
    event_bundle_sequence_id INTEGER, -- 이벤트 번들 순서 ID
    event_server_timestamp_offset INTEGER, -- 서버 타임스탬프 오프셋
    user_id TEXT, -- 사용자 ID
    user_pseudo_id TEXT, -- 가명 사용자 ID
    privacy_info BLOB, -- 개인정보 관련 정보 (JSON 데이터)
    user_properties BLOB, -- 사용자 속성 (JSON 데이터)
    user_first_touch_timestamp INTEGER, -- 사용자가 처음으로 상호작용한 타임스탬프
    user_ltv BLOB, -- 사용자의 수명 가치 정보 (JSON 데이터)
    device BLOB, -- 장치 정보 (JSON 데이터)
    geo BLOB, -- 지리 정보 (JSON 데이터)
    app_info BLOB, -- 앱 정보 (JSON 데이터)
    traffic_source BLOB, -- 트래픽 소스 정보 (JSON 데이터)
    stream_id TEXT, -- 스트림 ID
    platform TEXT, -- 플랫폼 (예: 웹, 모바일)
    event_dimensions BLOB, -- 이벤트 차원 (JSON 데이터)
    ecommerce BLOB, -- 전자상거래 정보 (JSON 데이터)
    items BLOB, -- 관련된 상품 정보 (JSON 데이터)
    collected_traffic_source BLOB, -- 수집된 트래픽 소스 (JSON 데이터)
    is_active_user BOOLEAN, -- 활성 사용자 여부
    batch_event_index INTEGER, -- 배치 이벤트 인덱스
    batch_page_id INTEGER, -- 배치 페이지 ID
    batch_ordering_id INTEGER, -- 배치 순서 ID
    session_traffic_source_last_click BLOB, -- 세션의 마지막 클릭 소스 (JSON 데이터)
    publisher BLOB -- 퍼블리셔 정보 (JSON 데이터) 
);

landing_report (
    event_date TEXT, -- 이벤트가 발생한 날짜
    landing_page TEXT, -- 랜딩 페이지 URL
    page_title TEXT, -- 랜딩 페이지 제목
    source_medium TEXT, -- 소스/미디엄 (예: google/cpc)
    source TEXT, -- 소스 (예: Google)
    medium TEXT, -- 미디엄 (예: cpc)
    campaign TEXT, -- 캠페인 이름
    content TEXT, -- 콘텐츠 이름
    term TEXT, -- 검색어
    source_platform TEXT, -- 소스 플랫폼 (예: 웹, 모바일)
    session TEXT, -- 세션 ID
    host_name TEXT, -- 호스트 이름 (예: example.com)
    user TEXT, -- 사용자 ID
    new_user TEXT, -- 신규 사용자 여부
    returning_user TEXT, -- 재방문 사용자 여부
    regular_purchase TEXT, -- 정기 구매 여부
    regular_user_id TEXT, -- 정기 구매 사용자 ID
    once_purchase TEXT, -- 일회성 구매 여부
    once_user_id TEXT, -- 일회성 구매 사용자 ID
    regular_value INTEGER, -- 정기 구매 총액
    once_value INTEGER -- 일회성 구매 총액 
);

Generate an appropriate SQL query based on the question.
You can Answer Only SQL query.

Question: 신규 사용자에 의한 특정 이벤트가 발생한 페이지 정보
    
"""

In [8]:
def parse_sql_response(response: str) -> str:
    # 'SQL 답변:' 이후의 텍스트만 추출
    sql_start = response.find("SQL 답변:")
    if sql_start == -1:
        return "SQL 답변을 찾을 수 없습니다."
    # 'SQL 답변:' 이후의 텍스트 반환
    return response[sql_start + len("SQL 답변:"):].strip()


In [9]:
response = llm_chain.run(question=question)
parsed_sql = parse_sql_response(response)
print(parsed_sql)

/DATA/acorn/anaconda3/envs/LangChain03Env/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


SELECT l.landing_page, l.page_title, l.source_medium, l.source, l.medium, l.campaign, l.content, l.term, l.source_platform, l.session, l.host_name, l.user, l.new_user, l.returning_user, l.regular_purchase, l.regular_user_id, l.once_purchase, l.once_user_id, l.regular_value, l.once_value FROM landing_report l JOIN events e ON l.user = e.user_id WHERE e.event_name = 'purchase' AND l.new_user = 'yes';


In [10]:
import sqlite3

# 데이터베이스 연결
conn = sqlite3.connect('test_database.db')
cursor = conn.cursor()

# 쿼리 실행
cursor.execute(parsed_sql)

# 결과 가져오기
result = cursor.fetchall()

# 결과 출력
for row in result:
    print(row)
# 연결 종료
conn.close()


('/home', 'Product Page', 'facebook/organic', 'facebook', 'organic', 'lead_gen', 'welcome', 'discount', 'mobile', 'session_92', 'example.com', 'user_17', 'yes', 'no', 'yes', 'regular_40', 'no', None, 8, 0)
('/checkout', 'Checkout Page', 'twitter/organic', 'twitter', 'organic', 'fall_campaign', 'new_post', 'holiday', 'desktop', 'session_7', 'example.com', 'user_43', 'yes', 'no', 'no', None, 'yes', 'once_30', 0, 34)
('/home', 'Home Page', 'google/cpc', 'google', 'cpc', 'winter_sale', 'offer', 'discount', 'desktop', 'session_2', 'shoponline.com', 'user_61', 'yes', 'no', 'no', None, 'no', None, 0, 0)
('/blog', 'Checkout Page', 'google/cpc', 'google', 'cpc', 'lead_gen', 'promo', 'blog', 'web', 'session_97', 'shoponline.com', 'user_14', 'yes', 'no', 'no', None, 'no', None, 0, 0)
('/home', 'Product Page', 'google/cpc', 'google', 'cpc', 'blog_launch', 'offer', 'blog', 'mobile', 'session_60', 'example.com', 'user_61', 'yes', 'no', 'no', None, 'no', None, 0, 0)
('/home', 'Checkout Page', 'google